In [1]:
import numpy as np
import networkx
from scipy import linalg, sparse
import sklearn
from sklearn.covariance import GraphicalLassoCV, GraphicalLasso
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
p=50

In [2]:
!pip install robust-selection --upgrade

Requirement already up-to-date: robust-selection in /Users/chautran/opt/anaconda3/lib/python3.7/site-packages (0.0.5)


## Define Functions

In [3]:
def MCC(omega_true, omega_hat, p):
    
    #Get off-diagonal entries
    omega_true = omega_true[np.tril_indices(p,-1)]
    omega_hat = omega_hat[np.tril_indices(p,-1)]
    
    #MCC
    TP = np.sum((omega_true!=0.) & (omega_hat!=0.))
    FN = np.sum((omega_true!=0.) & (omega_hat==0.))
    FP = np.sum((omega_true==0.) & (omega_hat!=0.))
    TN = np.sum((omega_true==0.) & (omega_hat==0.))
    MCC = (TP*TN - FP*FN)/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return MCC

## Create Precision matrix

In [4]:
#Create Omega
np.random.seed(1)
#edge probability 0.1 and p = 50
ER_graph = networkx.to_numpy_matrix(networkx.gnp_random_graph(n=p, p=0.1))

edge_weights = np.random.uniform(low=0.5, high=1., size=(p,p))
edge_signs = np.random.choice([-1,1], size=(p,p))

Omega = np.multiply(edge_weights, edge_signs)
Omega = np.multiply(ER_graph, Omega)
#Positive Definite procedure
for i in range(p):
    Omega[i,i] = 1
    off_sum = np.sum(np.abs(Omega[i,:i])) + np.sum(np.abs(Omega[i,i+1:]))
    Omega[i,:i] = Omega[i,:i]/(1.5*off_sum)
    Omega[i,i+1:] = Omega[i,i+1:]/(1.5*off_sum)
#Average for Symmetry
Omega = (Omega + Omega.T)/2
Sigma = np.linalg.inv(Omega)

In [5]:
#Omega

## Generate Data

In [6]:
n_list = [100, 250, 500]
n_datasets = 100
def generate_data(n):
    prng = np.random.RandomState(1)
    data = prng.multivariate_normal(np.zeros(p), Sigma, size=(n_datasets,n))
    return data
data_list = []
for n in n_list:
    data_list.append(generate_data(n))

## Graphical Lasso CV with Sklearn

In [7]:
lambda_cv = np.zeros((3,n_datasets))
mcc_cv = np.zeros((3,n_datasets))
for j in range (3):
    for i in range(n_datasets):
        #Fit model
        model = GraphicalLassoCV().fit(data_list[j][i])
        #Add lambda
        lambda_cv[j,i] = model.alpha_
        omega_hat = model.precision_
        mcc = MCC(Omega, omega_hat, p)
        mcc_cv[j,i] = mcc

/Users/chautran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/Users/chautran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/Users/chautran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/Users/chautran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


KeyboardInterrupt: 

In [ ]:
#mcc_cv

In [ ]:
#lambda_cv[0]

In [ ]:
robsel.robsel

In [ ]:
np.random

## RobSel

In [ ]:
from robsel.robsel import *

In [ ]:
#alpha_list = np.array([0.1, 0.21, 0.33, 0.44, 0.56])

lambda_robsel = np.zeros((3,n_datasets))
mcc_robsel = np.zeros((3,n_datasets))

#alpha = 0.9 for Figure 2
for j in range (3):
    for i in range(n_datasets):
        lambd = RobustSelection(data_list[j][i], B=200, alpha=0.9)
        lambda_robsel[j,i] = lambd

        #Fit Graphical Lasso with RobSel lambda
        model = GraphicalLasso(alpha=lambd).fit(data_list[j][i])
        omega_hat = model.precision_
        mcc = MCC(Omega, omega_hat, p)
        mcc_robsel[j,i] = mcc

## Plot

In [ ]:
# data1 = pd.DataFrame(lambda_cv.flatten(),columns=["lambd"])
# data1["type"] = np.full(300, "CV")
# data1["n"] = pd.Series(100*["n=100"] + 100*["n=250"] + 100*["n=500"]).astype('category')
# data1["mcc"] = mcc_cv.flatten()


data2 = pd.DataFrame(lambda_robsel.flatten(),columns=["lambd"])
data2["type"] = np.full(300, "RS")
data2["n"] = pd.Series(100*["n=100"] + 100*["n=250"] + 100*["n=500"]).astype('category')
data2["mcc"] = mcc_robsel.flatten()
data = pd.concat([data2], ignore_index=True)

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.lmplot(x="lambd",y='mcc', hue="n", data=data, order=2,scatter_kws={"s":10})
ax.axes[0,0].set_ylim(0,.6)
ax.axes[0,0].set_xlim(0.,0.4)

In [ ]:
plt.figure(figsize=(5.5, 3))
ax = sns.boxplot(x=data["lambd"], y=data["type"],hue=data["n"],orient="h")
ax.set_xlim(0,0.4)